# Example for 'custom' LED spectrum and custom constellation


In [1]:
# Import Transmitter
from vlc_rolling.transmitter import Transmitter
# Import Photodetector
# from vlc_rm.photodetector import Photodetector
# Import Indoor Environment
from vlc_rolling.indoorenv import Indoorenv

from vlc_rolling.imagesensor import Imagesensor
# Import REcursiveModel
# from vlc_rm.recursivemodel import Recursivemodel
# Import Symbol Constants
from vlc_rolling.constants import Constants as Kt

from vlc_rolling.sightpy import *

# Import numpy
import numpy as np

#Import luxpy   
import luxpy as lx

# Import Matplotlob
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [6, 6]
plt.rcParams['figure.facecolor']='white'

## Define transmitter, receiver and indoor space


In [2]:

green_wall = rgb(0.0, 1.0, 0.0)
red_wall = rgb(0.0, 1.0, 0.0)
white_wall = rgb(0.8, 0.8, 0.8)
floor_wall = rgb(0.1, 0.1, 0.1)

# Create indoor environment and 3d scene

room = Indoorenv(
    name="Matisse-CornellBox",
    size=[5, 5, 3],
    no_reflections=10,
    resolution=1/10,
    ceiling=('diffuse', white_wall),
    west=('diffuse', white_wall),
    north=('diffuse', green_wall),
    east=('diffuse', white_wall),
    south=('diffuse', red_wall),
    floor=('diffuse', floor_wall)
        )
room.create_environment()
print(room)

# Create a transmitter-type object
transmitter = Transmitter(
    room=room,
    name="Led1",
    led_type='gaussian',
    reference='RGB-Phosphor',
    position=[1.34, 0.8, 2.30],
    normal=[0, 0, -1],
    mlambert=1.3,
    wavelengths=[620, 530, 475],
    fwhm=[20, 30, 20],
    constellation='ieee16',
    luminous_flux=5000
            )
print(transmitter)

cam = Imagesensor(
    name="Camera",
    focal_length = 0.1,
    pixel_size = 1,
    image_height = 200,
    image_width = 200,
    camera_center = vec3(278, 278, 800),
    camera_look_at =  vec3(278, 278, 0),
    room = room,
    sensor='SonyStarvisBSI'
    )
print(cam)

room.render_environment()


 List of parameters for indoor envirionment Matisse-CornellBox: 
Name: Matisse-CornellBox
Size [x y z] -> [m]: [5.0000e+00 5.0000e+00 3.0000e+00] 
Order reflection: 10 
Resolution points [m]: 0.10000000149011612
Smaller Area [m^2]: Non defined, create grid.
Number of points: Non defined, create grid.


 List of parameters for LED transmitter: 
Name: Led1
Position [x y z]: [1.3400e+00 8.0000e-01 2.3000e+00] 
Normal Vector [x y z]: [[0.0000e+00 0.0000e+00 -1.0000e+00]] 
Lambert Number: 1.2999999523162842 
Central Wavelengths [nm]: [6.2000e+02 5.3000e+02 4.7500e+02] 
FWHM [nm]: [2.0000e+01 3.0000e+01 2.0000e+01]
Luminous Flux [lm]: 5000.0
Correlated Color Temperature: [[-4.3902e+06]]
CIExy coordinates: [[2.3633e-01 1.9580e-01 5.6787e-01]]
ILER [W/lm]: 
 [[3.8001e-03 0.0000e+00 0.0000e+00]
 [0.0000e+00 1.8197e-03 0.0000e+00]
 [0.0000e+00 0.0000e+00 1.1960e-02]] 
Average Power per Channel Color: 
 [1.2667e-03 6.0655e-04 3.9868e-03] 
Total Power emmited by the Transmitter [W]: 
 29.30032767

ValueError: not enough values to unpack (expected 2, got 0)